# Inspección y Diagnóstico (.dtypes y .info)

Teoría: Antes de tocar nada, debemos saber qué interpreta Pandas.

In [7]:
import pandas as pd 
import numpy as np

In [8]:
#Creamos un DataFrame con variedad de tipos
df=pd.DataFrame({
    'Nombre': ['Ana', 'Luis', 'Pedro'], # Texto -> object
    'Edad': [25, 30, 22],   # Entero -> int64
    'Altura': [1.65,1.80,1.75], # Decimal -> float64
    'Es_Estudiante':[True,False, True], # Booleano -> bool
    'Fecha_Ingreso':['2023-01-01', '2023-02-15', '2023-03-20'] # Texto (aún) -> object
})

print("--- Tipos Detectados Automáticamente ---")
print(df.dtypes)

print("\n--- Informe de Memoria y Nulos ---")
df.info()


--- Tipos Detectados Automáticamente ---
Nombre               str
Edad               int64
Altura           float64
Es_Estudiante       bool
Fecha_Ingreso        str
dtype: object

--- Informe de Memoria y Nulos ---
<class 'pandas.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Nombre         3 non-null      str    
 1   Edad           3 non-null      int64  
 2   Altura         3 non-null      float64
 3   Es_Estudiante  3 non-null      bool   
 4   Fecha_Ingreso  3 non-null      str    
dtypes: bool(1), float64(1), int64(1), str(2)
memory usage: 231.0 bytes


## Conversión Explícita (.astype)

Teoría: El dato viene mal formato (números como texto) y hay que forzarlos

In [9]:
#Simulación de error común: Números cargados como texto (fíjate en las comillas)
precios = pd.Series(["1000", "2000", "3000"])
print("--- Error Típico: Suma de Textos ---")
print("Suma original:", precios.sum()) # Resultado: "100020003000"
print("tipo:", precios.dtype)

#Solución: Casting a Entero
precios_num=precios.astype(int)
print("\n--- Solución: Suma Numérica ---")
print("Suma corregida:", precios_num.sum()) # Resultado: 6000
print("Nuevo tipo:", precios_num.dtype)


--- Error Típico: Suma de Textos ---
Suma original: 100020003000
tipo: str

--- Solución: Suma Numérica ---
Suma corregida: 6000
Nuevo tipo: int64


# Conversión Inteligente (pd.to_numeric)

Teoría: astype falla si hay basura en los datos. to_numeric es más robusto y maneja errores.

In [10]:
#Dato Sucio: Alguien escribio "Error" en lugar de un número
s_sucia= pd.Series(["10.5", "20.2", "Error", "30.1"])

# Si usas s_sucia.astype(float) -> Python explotará y dará error.

## Solución: Usar to_numeric con 'coerce' (forzar error a NaN)
s_limpia=pd.to_numeric(s_sucia, errors='coerce')

print("--- Limpieza de Datos Numéricos ---")
print(s_limpia)
# Fíjate que "Error" se convirtió en NaN (Not a Number) y el resto en float.

--- Limpieza de Datos Numéricos ---
0    10.5
1    20.2
2     NaN
3    30.1
dtype: float64


## Manejo de Fechas (pd.to_datetime)

Teoría: Convertir texto a objetos de tiempo reales para extraer información.

In [11]:
#Usamos la columna 'Fecha_Ingreso' del primer ejemplo que era object
print("Original:",df['Fecha_Ingreso'].dtype)

#Convertimos a datetime
df['Fecha_Ingreso']=pd.to_datetime(df['Fecha_Ingreso'])
print("Convertido:", df['Fecha_Ingreso'].dtype)

#Ahora podemos hacer magia temporal
print("\n--- Extracción de Meses ---")
print(df['Fecha_Ingreso'].dt.month_name())

Original: str
Convertido: datetime64[us]

--- Extracción de Meses ---
0     January
1    February
2       March
Name: Fecha_Ingreso, dtype: str


# Optimización con Categorías (category)

Teoría: Ahorrar memoria convirtiendo textos repetitivos en categorías numéricas internas.

In [12]:
#Creamos una serie gigante repetitiva
colores=pd.Series(["Rojo","Azul","Verde","Rojo"]*1000)

#Medimos memoria como 'object' (texto)
mem_obj=colores.memory_usage(deep=True)

#Convertimos a 'category'
colores_cat=colores.astype('category')
mem_cat=colores_cat.memory_usage(deep=True)

print(f"--- Ahorro de Memoria ---")
print(f"Memoria Object: {mem_obj} bytes")
print(f"Memoria Category: {mem_cat} bytes")
print(f"Reducción: {round((1 - mem_cat/mem_obj) * 100, 2)}%")

--- Ahorro de Memoria ---
Memoria Object: 213132 bytes
Memoria Category: 4292 bytes
Reducción: 97.99%


# Selección por Tipos (select_dtypes)

Teoría: Filtrar columnas no por nombre, sino por su naturaleza.

In [13]:
# Del DataFrame original 'df', queremos solo las columnas numéricas
df_numerico=df.select_dtypes(include=['number'])

print("--- Solo Columnas Numéricas ---")
print(df_numerico.head())

--- Solo Columnas Numéricas ---
   Edad  Altura
0    25    1.65
1    30    1.80
2    22    1.75
